In [38]:
!pip3 install -q selenium bs4
from selenium import webdriver
from requests import Session, Request
import pickle
from bs4 import BeautifulSoup as bs
import time
from pathlib import Path

In [ ]:
file_object = open("user.pkl", 'ab')
parentPageUrl = 'http://www.tsetmc.com/Loader.aspx?ParTree=15131F'
saveLocation = "../xcels/"

session = Session()
request = Request("Get", parentPageUrl)
prepared = session.prepare_request(request)
respond = session.send(prepared, verify=False)

startYear = 1398
endYear = 1399
monthDays = [31, 31, 31, 31, 31, 31, 30, 30, 30, 30, 30, 29]

for year in range(startYear, endYear):
    for month in range(12):
        for day in range(monthDays[month]):
#             print(str(year) + "-" + str(month+1) + "-" + str(day+1))
            if not Path(saveLocation + str(year) + "-" + str(month+1) + "-" + str(day+1) + ".xlsx").is_file():
                with open(saveLocation + str(year) + "-" + str(month+1) + "-" + str(day+1) +
                          ".xlsx", 'wb') as f:
                    xcelUrl = "http://members.tsetmc.com/tsev2/excel/MarketWatchPlus.aspx?d=" + \
                        str(year) + "/" + str(month+1) + "/" + str(day+1)
                    xcelRequest = Request("Get", xcelUrl)
                    xcelPrepared = session.prepare_request(xcelRequest)
                    xcelRespond = session.send(xcelPrepared, verify=False, stream=True)
                    xcelRespond.raise_for_status()
                    for chunk in xcelRespond.iter_content(chunk_size=8192): 
                        if chunk: # filter out keep-alive new chunks
                            f.write(chunk)
                        # f.flush()
                    time.sleep(30)
print(xcelRespond.cookies)



In [ ]:
# # pickle.dump(requests.utils.dict_from_cookiejar(resp.cookies), file_object)

# soup = bs(resp.text, 'html.parser')
# name = soup.find_all(itemprop='name')
# name = [x.get_text() for x in name]
# moredis = soup.findAll("p", {"class": "prd-desc more"})
# moredis = [x.get_text() for x in moredis]
# weight = soup.findAll("span", {"class": "unit"})
# weight = [x.get_text() for x in weight]

# import datetime
# now = datetime.datetime.now()

# with open('somefile') as f:
#     cookies = requests.utils.cookiejar_from_dict(pickle.load(f))
#     session = requests.session(cookies=cookies)

# from http.cookies import SimpleCookie
# rawdata = 'Cookie: _ga=GA1.2.969355752.1534482039;' \
#           ' JSESSIONID=nM0Rmitqyzs4sWMMdOBXB5BDbq09pS3ilLqwUJOV1bCZv_BxQgHS!484861232'
# cookie = SimpleCookie()
# cookie.load(rawdata)
